In [1]:
#1_1 - 3_2
import pygame
import math

pygame.init()

WIDTH = 800
HEIGHT = 600
screen = pygame.display.set_mode(size=(WIDTH, HEIGHT))

clock = pygame.time.Clock()

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (0, 0, 255)

font = pygame.font.Font(None, size=24)

robot_x = 0
robot_y = 0
robot_direction = 0 # градусы, 0 = вправо

# linear_speed = 50 # пикселей/секунду
# angular_speed = 30  # градусов/секунду

speed_multiplier = 1.0

control_mode = "keyboard"

def draw_robot(screen, x, y, direction, radius):
    """Рисует робота с направлением"""
    
    screen_x = x + WIDTH // 2
    screen_y = HEIGHT // 2 - y
    
    pygame.draw.circle(screen, color=BLUE, center=(screen_x, screen_y), radius=radius)
    
    angle = math.radians(direction)
    
    tip_x = screen_x + math.cos(angle) * radius * 1.5
    tip_y = screen_y - math.sin(angle) * radius * 1.5
    
    left_angle = angle + math.radians(140)
    left_x = screen_x + math.cos(left_angle) * radius * 0.8
    left_y = screen_y - math.sin(left_angle) * radius * 0.8
    
    right_angle = angle - math.radians(140)
    right_x = screen_x + math.cos(right_angle) * radius * 0.8
    right_y = screen_y - math.sin(right_angle) * radius * 0.8
    
    pygame.draw.polygon(screen, WHITE, [
        (tip_x, tip_y),
        (left_x, left_y),
        (right_x, right_y)
    ])
    
    pygame.draw.circle(surface=screen, color=BLACK, center=(screen_x, screen_y), radius=radius, width=2)

def draw_info(screen, x, y, direction, linear_speed, angular_speed, control_mode):
    """Отображает информацию о роботе на экране"""
    # TODO: Создайте список строк с информацией
    lines = [
        "Информация:", # Подсказка: используйте f-строки
        f"Позиция: ({int(x)}, {int(y)})",# Позиция: округлите x и y до целых через int()
        f"Направление: {int(direction % 360)}", # Направление: нормализуйте через % 360
        f"Линейная скорость: {int(linear_speed)} пикс/с",
        f"Угловая скорость: {int(angular_speed)} град/с",
        # TODO: Добавьте строку с режимом
        f"Режим: {control_mode}",
        "Tab - переключить режим, R - сброс"
    ]
    
    y_offset = 10
    for line in lines:
        text = font.render(line, True, BLACK)
        screen.blit(source=text, dest=(10, y_offset))
        y_offset += 25
        pass

def draw_trail(screen, trail):
    """Рисует след за роботом"""
    
    if len(trail) < 2:
        return  # Нечего рисовать
    
    for i in range(len(trail) - 1):
        point1 = trail[i]
        point2 = trail[i + 1]
        
        screen_x1 = (point1[0]) + WIDTH // 2
        screen_y1 = HEIGHT // 2 - point1[1]
        screen_x2 = (point2[0]) + WIDTH // 2
        screen_y2 = HEIGHT // 2 - point2[1]
        
        pygame.draw.line(
            surface=screen,
            color=(150, 150, 255),  # Светло-синий
            start_pos=(screen_x1, screen_y1),
            end_pos=(screen_x2, screen_y2),
            width=2
        )

MAX_X = WIDTH // 2 - 20  # Отступ 20 пикселей от края
MAX_Y = HEIGHT // 2 - 20

bar_width = 100
bar_height = 20
bar_x = WIDTH - bar_width - 10
bar_y = 10

trail = []
MAX_TRAIL_LENGTH = 100

# Игровой цикл
running = True
while running:

    linear_speed = 0
    angular_speed = 0
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 1:  # Левая кнопка мыши
                mouse_screen_x, mouse_screen_y = event.pos
        
                robot_x = mouse_screen_x - WIDTH // 2
                robot_y = HEIGHT // 2 - mouse_screen_y

                #print(f"Телепортация в ({int(robot_x)}, {int(robot_y)})")

            if event.button == 2:  # Средняя кнопка
                robot_x = 0
                robot_y = 0
                #print("Возврат в центр")

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_TAB:
                if control_mode == "keyboard":
                    control_mode = "mouse"
                    print("Режим: следование за мышью")
                else:
                    control_mode = "keyboard"
                    print("Режим: управление клавиатурой")

            if event.key == pygame.K_r:
                robot_x = 0
                robot_y = 0
                robot_direction = 0
                trail.clear()
                print("Позиция сброшена")

            if event.key == pygame.K_EQUALS:
                speed_multiplier += 0.1 # Например, 0.25
                # TODO: Ограничьте максимум (не больше 3.0)
                speed_multiplier = min(3, speed_multiplier)
                print(f"Скорость: {speed_multiplier * 100:.0f}%")

            # TODO: Уменьшение скорости по клавише -
            elif event.key == pygame.K_MINUS:
                speed_multiplier -= 0.1  # Например, 0.25
                # TODO: Ограничьте минимум (не меньше 0.25)
                speed_multiplier = max(0.1, speed_multiplier)
                print(f"Скорость: {speed_multiplier * 100:.0f}%")
                        

    if control_mode == "keyboard":

        keys = pygame.key.get_pressed()

        #speed_multiplier = 0.5 if keys[pygame.K_LSHIFT] else 1.0

        if keys[pygame.K_w] and keys[pygame.K_s]:
            linear_speed = 0
        elif keys[pygame.K_w]:
            linear_speed = 150 * speed_multiplier
        elif keys[pygame.K_s]:
            linear_speed = -100 * speed_multiplier

        if keys[pygame.K_a]:
            angular_speed += 120 * speed_multiplier
        if keys[pygame.K_d]:
            angular_speed += -120 * speed_multiplier

    
    elif control_mode == "mouse":
        # Код следования за мышью
        mouse_screen_x, mouse_screen_y = pygame.mouse.get_pos()

        target_x = mouse_screen_x - WIDTH // 2
        target_y = HEIGHT // 2 - mouse_screen_y

        dx = target_x - robot_x
        dy = target_y - robot_y

        distance = math.hypot(dx, dy)

        angle_to_target_rad = math.atan2(dy, dx)
        angle_to_target_deg = math.degrees(angle_to_target_rad)
        angle_difference = angle_to_target_deg - robot_direction

        angle_difference %= 360

        if abs(angle_difference) > 5:
            k = speed_multiplier
            if angle_difference > 0:
                angular_speed = k * angle_difference
            else:
                angular_speed = -k * angle_difference
        else:
            angular_speed = 0

        if distance > 5 and abs(angle_difference) < 30:
            linear_speed = 150 * speed_multiplier  # Например, 150
        else:
            linear_speed = 0
     
    dt = clock.tick(60) / 1000.0

    robot_direction += angular_speed * dt
    angle_rad = math.radians(robot_direction)
    robot_x += math.cos(angle_rad)*linear_speed*dt
    robot_y += math.sin(angle_rad)*linear_speed*dt

    if robot_x > MAX_X:
        robot_x = MAX_X
    elif robot_x < -MAX_X:
        robot_x = -MAX_X

    if robot_y > MAX_Y:
        robot_y = MAX_Y
    elif robot_y < -MAX_Y:
        robot_y = -MAX_Y

    trail.append((robot_x, robot_y))
    if len(trail) > MAX_TRAIL_LENGTH:
        trail.pop(0)
    
    screen.fill(WHITE)

    #print(f" Координаты: ({robot_x}, {robot_y})")

    draw_trail(screen, trail)

    pygame.draw.rect(surface=screen, color=(100, 100, 100), rect=(bar_x, bar_y, bar_width, bar_height))
    fill_width = int(bar_width * (speed_multiplier / 3.0))
    pygame.draw.rect(surface=screen, color=(0, 200, 0), rect=(bar_x, bar_y, fill_width, bar_height))
    pygame.draw.rect(surface=screen, color=BLACK, rect=(bar_x, bar_y, bar_width, bar_height), width=2)
    
    draw_robot(screen=screen, x=robot_x, y=robot_y, direction=robot_direction, radius=15)
    draw_info(screen, robot_x, robot_y, robot_direction, linear_speed, angular_speed, control_mode)
    
    pygame.display.flip()
   

pygame.quit()

pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
Режим: следование за мышью
Режим: управление клавиатурой


In [98]:
#3_3
import pygame
import math
import random

pygame.init()

WIDTH = 800
HEIGHT = 600
screen = pygame.display.set_mode(size=(WIDTH, HEIGHT))

clock = pygame.time.Clock()

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (0, 0, 255)

font = pygame.font.Font(None, size=24)

robot_x = 0
robot_y = 0
robot_direction = 0 # градусы, 0 = вправо

speed_multiplier = 1.0

control_mode = "keyboard"

def draw_robot(screen, x, y, direction, radius):
    """Рисует робота с направлением"""
    
    screen_x = x + WIDTH // 2
    screen_y = HEIGHT // 2 - y
    
    pygame.draw.circle(screen, color=BLUE, center=(screen_x, screen_y), radius=radius)
    
    angle = math.radians(direction)
    
    tip_x = screen_x + math.cos(angle) * radius * 1.5
    tip_y = screen_y - math.sin(angle) * radius * 1.5
    
    left_angle = angle + math.radians(140)
    left_x = screen_x + math.cos(left_angle) * radius * 0.8
    left_y = screen_y - math.sin(left_angle) * radius * 0.8
    
    right_angle = angle - math.radians(140)
    right_x = screen_x + math.cos(right_angle) * radius * 0.8
    right_y = screen_y - math.sin(right_angle) * radius * 0.8
    
    pygame.draw.polygon(screen, WHITE, [
        (tip_x, tip_y),
        (left_x, left_y),
        (right_x, right_y)
    ])
    
    pygame.draw.circle(surface=screen, color=BLACK, center=(screen_x, screen_y), radius=radius, width=2)

def draw_info(screen, x, y, direction, linear_speed, angular_speed, control_mode, score, time):
    """Отображает информацию о роботе на экране"""
    # TODO: Создайте список строк с информацией
    lines = [
        "Информация:", # Подсказка: используйте f-строки
        f"Счет: {score}",
        f"Время: {int(time)}с",
        f"Позиция: ({int(x)}, {int(y)})",# Позиция: округлите x и y до целых через int()
        f"Направление: {int(direction % 360)}", # Направление: нормализуйте через % 360
        f"Линейная скорость: {int(linear_speed)} пикс/с",
        f"Угловая скорость: {int(angular_speed)} град/с",
        # TODO: Добавьте строку с режимом
        f"Режим: {control_mode}",
        "Tab - переключить режим, R - сброс"
    ]
    
    y_offset = 10
    for line in lines:
        text = font.render(line, True, BLACK)
        screen.blit(source=text, dest=(10, y_offset))
        y_offset += 25
        pass

def draw_trail(screen, trail):
    """Рисует след за роботом"""
    
    if len(trail) < 2:
        return  # Нечего рисовать
    
    for i in range(len(trail) - 1):
        point1 = trail[i]
        point2 = trail[i + 1]
        
        screen_x1 = (point1[0]) + WIDTH // 2
        screen_y1 = HEIGHT // 2 - point1[1]
        screen_x2 = (point2[0]) + WIDTH // 2
        screen_y2 = HEIGHT // 2 - point2[1]
        
        pygame.draw.line(
            surface=screen,
            color=(150, 150, 255),  # Светло-синий
            start_pos=(screen_x1, screen_y1),
            end_pos=(screen_x2, screen_y2),
            width=2
        )

def spawn_coin():
    """Создаёт новую точку в случайном месте"""
    
    x = random.randint(-MAX_X, MAX_X)  
    y = random.randint(-MAX_Y, MAX_Y) 
    
    color = random.choice(COIN_COLORS)
    
    coin = {
        'x': x,
        'y': y,
        'color': color,
        'radius': COIN_RADIUS
    }
    
    return coin

def draw_coins(screen, coins):
    """Рисует все точки на экране"""
    for coin in coins:
        screen_x = coin['x'] + WIDTH//2
        screen_y = HEIGHT//2 - coin['y']
        
        pygame.draw.circle(
            surface=screen,
            color=coin['color'],
            center=(screen_x, screen_y),
            radius=coin['radius']
        )

def restart_game():
    """Перезапускает игру"""
    global score, coins, game_start_time, robot_x, robot_y, robot_direction, trail
    
    score = 0
    
    coins.clear()
    
    for i in range(5):
        coins.append(spawn_coin())
    
    robot_x = 0
    robot_y = 0
    robot_direction = 0
    
    trail.clear()
    
    game_start_time = pygame.time.get_ticks()
    
    print("Игра перезапущена!")

obs_radius = 10
obs_color = (100, 100, 100)
obsts = []
    
for i in range(6):

    x = random.randint(-MAX_X, MAX_X)  
    y = random.randint(-MAX_Y, MAX_Y) 
    
    obs = {
        'x': x,
        'y': y,
        'color': obs_color,
        'radius': obs_radius
    }
    obsts.append(obs)

MAX_X = WIDTH // 2 - 20  # Отступ 20 пикселей от края
MAX_Y = HEIGHT // 2 - 20

bar_width = 100
bar_height = 20
bar_x = WIDTH - bar_width - 10
bar_y = 10

trail = []
MAX_TRAIL_LENGTH = 100

coins = []
score = 0

COIN_COLORS = [
    (255, 200, 0),   # Золотой
    (0, 200, 255),   # Голубой
    (255, 100, 200), # Розовый
    (150, 255, 100)  # Зелёный
]

COIN_RADIUS = 8
ROBOT_RADIUS = 15

for i in range(5):  
    coins.append(spawn_coin())

game_start_time = pygame.time.get_ticks()
GAME_DURATION = 60

# Игровой цикл
running = True
while running:

    elapsed_time = (pygame.time.get_ticks() - game_start_time) / 1000 

    remaining_time = GAME_DURATION - elapsed_time

    if remaining_time <= 0:
        print(f"Игра окончена! Финальный счёт: {score}")

        if remaining_time <= -10:
            remaining_time = 0
            restart_game()


    linear_speed = 0
    angular_speed = 0
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 1:  # Левая кнопка мыши
                mouse_screen_x, mouse_screen_y = event.pos
        
                robot_x = mouse_screen_x - WIDTH // 2
                robot_y = HEIGHT // 2 - mouse_screen_y

                #print(f"Телепортация в ({int(robot_x)}, {int(robot_y)})")

            if event.button == 2:  # Средняя кнопка
                robot_x = 0
                robot_y = 0
                #print("Возврат в центр")

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_TAB:
                if control_mode == "keyboard":
                    control_mode = "mouse"
                    print("Режим: следование за мышью")
                else:
                    control_mode = "keyboard"
                    print("Режим: управление клавиатурой")

            if event.key == pygame.K_r:
                restart_game()

            if event.key == pygame.K_EQUALS:
                speed_multiplier += 0.1 # Например, 0.25
                # TODO: Ограничьте максимум (не больше 3.0)
                speed_multiplier = min(3, speed_multiplier)
                print(f"Скорость: {speed_multiplier * 100:.0f}%")

            # TODO: Уменьшение скорости по клавише -
            elif event.key == pygame.K_MINUS:
                speed_multiplier -= 0.1  # Например, 0.25
                # TODO: Ограничьте минимум (не меньше 0.25)
                speed_multiplier = max(0.1, speed_multiplier)
                print(f"Скорость: {speed_multiplier * 100:.0f}%")
                        

    if control_mode == "keyboard":

        keys = pygame.key.get_pressed()

        #speed_multiplier = 0.5 if keys[pygame.K_LSHIFT] else 1.0

        if keys[pygame.K_w] and keys[pygame.K_s]:
            linear_speed = 0
        elif keys[pygame.K_w]:
            linear_speed = 150 * speed_multiplier
        elif keys[pygame.K_s]:
            linear_speed = -100 * speed_multiplier

        if keys[pygame.K_a]:
            angular_speed += 120 * speed_multiplier
        if keys[pygame.K_d]:
            angular_speed += -120 * speed_multiplier

    
    elif control_mode == "mouse":
        # Код следования за мышью
        mouse_screen_x, mouse_screen_y = pygame.mouse.get_pos()

        target_x = mouse_screen_x - WIDTH // 2
        target_y = HEIGHT // 2 - mouse_screen_y

        dx = target_x - robot_x
        dy = target_y - robot_y

        distance = math.hypot(dx, dy)

        angle_to_target_rad = math.atan2(dy, dx)
        angle_to_target_deg = math.degrees(angle_to_target_rad)
        angle_difference = angle_to_target_deg - robot_direction

        angle_difference %= 360

        if abs(angle_difference) > 5:
            k = speed_multiplier
            if angle_difference > 0:
                angular_speed = k * angle_difference
            else:
                angular_speed = -k * angle_difference
        else:
            angular_speed = 0

        if distance > 5 and abs(angle_difference) < 30:
            linear_speed = 150 * speed_multiplier  # Например, 150
        else:
            linear_speed = 0
     
    dt = clock.tick(60) / 1000.0

    robot_direction += angular_speed * dt
    angle_rad = math.radians(robot_direction)
    robot_x += math.cos(angle_rad)*linear_speed*dt
    robot_y += math.sin(angle_rad)*linear_speed*dt

    for obs in obsts:
        dx = obs['x'] - robot_x
        dy = obs['y'] - robot_y
        distance = math.hypot(dx, dy)
        if distance < ROBOT_RADIUS + obs['radius']:
            # if linear_speed >= 0:
            #     linear_speed = -abs(linear_speed)
            # else:
            #     linear_speed = abs(linear_speed)
            robot_x -= math.cos(angle_rad)*linear_speed*dt
            robot_y -= math.sin(angle_rad)*linear_speed*dt
            pass

    if robot_x > MAX_X:
        robot_x = MAX_X
    elif robot_x < -MAX_X:
        robot_x = -MAX_X

    if robot_y > MAX_Y:
        robot_y = MAX_Y
    elif robot_y < -MAX_Y:
        robot_y = -MAX_Y

    trail.append((robot_x, robot_y))
    if len(trail) > MAX_TRAIL_LENGTH:
        trail.pop(0)

    coins_to_remove = [] 

    for coin in coins:
        dx = coin['x'] - robot_x
        dy = coin['y'] - robot_y
        distance = math.hypot(dx, dy)
        
        if distance < ROBOT_RADIUS + coin['radius']:
            coins_to_remove.append(coin)
            score += 1

    for coin in coins_to_remove:
        coins.remove(coin)

    if len(coins) < 5:
        coins.append(spawn_coin())
    
    screen.fill(WHITE)

    for obs in obsts:
        screen_x = obs['x'] + WIDTH // 2
        screen_y = HEIGHT // 2 - obs['y']
        pygame.draw.circle(
            surface=screen,
            color=(100, 100, 100),
            center=(screen_x, screen_y),
            radius=obs['radius']
        )

    draw_trail(screen, trail)

    pygame.draw.rect(surface=screen, color=(100, 100, 100), rect=(bar_x, bar_y, bar_width, bar_height))
    fill_width = int(bar_width * (speed_multiplier / 3.0))
    pygame.draw.rect(surface=screen, color=(0, 200, 0), rect=(bar_x, bar_y, fill_width, bar_height))
    pygame.draw.rect(surface=screen, color=BLACK, rect=(bar_x, bar_y, bar_width, bar_height), width=2)
    
    draw_robot(screen=screen, x=robot_x, y=robot_y, direction=robot_direction, radius=15)
    draw_coins(screen, coins)
    draw_info(screen, robot_x, robot_y, robot_direction, linear_speed, angular_speed, control_mode, score, remaining_time)

    if remaining_time <= 0:

        if score > 0:
            pygame.draw.rect(screen, (130, 203, 69), (300, 250, 200, 80))
            pygame.draw.rect(screen, (76, 187, 203), (300, 250, 200, 80), width=5)
            textf1 = font.render(f"Игра окончена!", True, BLACK)
            textf2 = font.render(f"Финальный счёт: {score}", True, BLACK)
            screen.blit(textf1, (340, 270))
            screen.blit(textf2, (320, 294))

        else:
            pygame.draw.rect(screen, (255, 50, 60), (300, 250, 200, 80))
            pygame.draw.rect(screen, (150, 0, 24), (300, 250, 200, 80), width=5)
            textf1 = font.render(f"Игра окончена!", True, BLACK)
            textf2 = font.render(f"Финальный счёт: {score}", True, BLACK)
            screen.blit(textf1, (340, 270))
            screen.blit(textf2, (320, 294))

    
    pygame.display.flip()
   

pygame.quit()

Режим: следование за мышью
Игра окончена! Финальный счёт: 10
Игра окончена! Финальный счёт: 10
Игра окончена! Финальный счёт: 10
Игра окончена! Финальный счёт: 10
Игра окончена! Финальный счёт: 10
Игра окончена! Финальный счёт: 10
Игра окончена! Финальный счёт: 10
Игра окончена! Финальный счёт: 10
Игра окончена! Финальный счёт: 10
Игра окончена! Финальный счёт: 10
Игра окончена! Финальный счёт: 10
Игра окончена! Финальный счёт: 10
Игра окончена! Финальный счёт: 10
Игра окончена! Финальный счёт: 10
Игра окончена! Финальный счёт: 10
Игра окончена! Финальный счёт: 10
Игра окончена! Финальный счёт: 10
Игра окончена! Финальный счёт: 10
Игра окончена! Финальный счёт: 10
Игра окончена! Финальный счёт: 10
Игра окончена! Финальный счёт: 10
Игра окончена! Финальный счёт: 10
Игра окончена! Финальный счёт: 10
Игра окончена! Финальный счёт: 10
Игра окончена! Финальный счёт: 10
Игра окончена! Финальный счёт: 10
Игра окончена! Финальный счёт: 10
Игра окончена! Финальный счёт: 10
Игра окончена! Финаль